In [73]:
#r "nuget:System.Device.Gpio,1.1.0-prerelease.20276.1"
#r "nuget:IoT.Device.Bindings,1.1.0-prerelease.20276.1 "
#r "nuget:UnitsNet,4.68.0"

In [74]:
using Iot.Device.SenseHat;
using System;
using System.Drawing;
using (var ledMatrix = new SenseHatLedMatrixI2c())
{
        ledMatrix.Fill(Color.Black);
}

In [75]:
async IAsyncEnumerable<double> ReadTemperatureAsync(int count = 10){   
    using (var sh = new SenseHat())
    {
        for (var i = 0; i < count; i++){
            yield return (sh.Temperature2.DegreesFahrenheit);
            await Task.Delay(60000);
        }
    };
}

In [76]:
using System;
var temperatures = new List<(DateTime dt, double val)>();


In [77]:
DateTime RoundUp(DateTime dt, TimeSpan d)
{
    return new DateTime((dt.Ticks + d.Ticks - 1) / d.Ticks * d.Ticks, dt.Kind);
}

In [78]:
await foreach (var result in ReadTemperatureAsync(15))
{
    var t = RoundUp(DateTime.Now, TimeSpan.FromMinutes(1));
    Console.WriteLine(t + " : " + result);
    temperatures.Add((t, result));
}

8/15/2020 6:59:00 PM : 101.35999908447265
8/15/2020 7:00:00 PM : 101.40500183105468
8/15/2020 7:01:00 PM : 101.5324981689453
8/15/2020 7:02:00 PM : 101.50999679565429
8/15/2020 7:03:00 PM : 101.40875091552734
8/15/2020 7:04:00 PM : 101.37500228881835
8/15/2020 7:05:00 PM : 101.39374771118163
8/15/2020 7:06:00 PM : 101.34500274658203
8/15/2020 7:07:00 PM : 101.30374908447264
8/15/2020 7:08:00 PM : 101.38250045776365
8/15/2020 7:09:00 PM : 101.31875228881835
8/15/2020 7:10:00 PM : 101.31875228881835
8/15/2020 7:11:00 PM : 101.15750045776366
8/15/2020 7:12:00 PM : 101.14624633789062
8/15/2020 7:13:00 PM : 101.13125


In [79]:
#r "nuget:Microsoft.Azure.CognitiveServices.AnomalyDetector"

In [80]:
using Microsoft.Azure.CognitiveServices.AnomalyDetector;
using Microsoft.Azure.CognitiveServices.AnomalyDetector.Models;
using System.Collections;
using Point = Microsoft.Azure.CognitiveServices.AnomalyDetector.Models.Point;

Request GetSeries(List<(DateTime dt, double val)> list)
{
    var points = list
        // .TakeLast(take)
        .Select(e => new Point(e.dt,e.val)).ToList();
    Console.WriteLine(points);
    return new Request(points, Granularity.Minutely, period: 1); 
}

In [81]:
async Task<bool> LastDetectSampleAsync(IAnomalyDetectorClient client, Request request)
{

    Console.WriteLine("Detecting the anomaly status of the latest point in the series.");
    LastDetectResponse result = await client.LastDetectAsync(request);
    return result.IsAnomaly;
}

In [82]:
using System;
using System.IO;
using System.Text;

string path = "config.json";
string readText = File.ReadAllText(path);
var dict = System.Text.Json.JsonSerializer.Deserialize<Dictionary<string, string>>(readText);

string endpoint = dict["endpoint"];
string key = dict["subscriptionkey"];


In [83]:
IAnomalyDetectorClient client = new AnomalyDetectorClient(new ApiKeyServiceClientCredentials(key))
{
    Endpoint = endpoint
};

In [86]:
foreach (var temp in temperatures)
{
    Console.WriteLine(temp.dt + " : " + temp.val);
}
var lastTemp = temperatures.Last();
temperatures.Add((lastTemp.dt.AddMinutes(1) , 200));

var request = GetSeries(temperatures); // The request payload with points from the data file
var isAnomaly = await LastDetectSampleAsync(client, request);
Console.WriteLine($"Anomaly {isAnomaly}");
if (isAnomaly)
{
    using (var ledMatrix = new SenseHatLedMatrixI2c())
    {
            ledMatrix.Fill(Color.Red);
    }
}


8/15/2020 6:59:00 PM : 101.35999908447265
8/15/2020 7:00:00 PM : 101.40500183105468
8/15/2020 7:01:00 PM : 101.5324981689453
8/15/2020 7:02:00 PM : 101.50999679565429
8/15/2020 7:03:00 PM : 101.40875091552734
8/15/2020 7:04:00 PM : 101.37500228881835
8/15/2020 7:05:00 PM : 101.39374771118163
8/15/2020 7:06:00 PM : 101.34500274658203
8/15/2020 7:07:00 PM : 101.30374908447264
8/15/2020 7:08:00 PM : 101.38250045776365
8/15/2020 7:09:00 PM : 101.31875228881835
8/15/2020 7:10:00 PM : 101.31875228881835
8/15/2020 7:11:00 PM : 101.15750045776366
8/15/2020 7:12:00 PM : 101.14624633789062
8/15/2020 7:13:00 PM : 101.13125
8/15/2020 7:14:00 PM : 200
System.Collections.Generic.List`1[Microsoft.Azure.CognitiveServices.AnomalyDetector.Models.Point]
Detecting the anomaly status of the latest point in the series.
Anomaly True


In [85]:
using Iot.Device.SenseHat;
using System;
using System.Drawing;
using (var ledMatrix = new SenseHatLedMatrixI2c())
{
        ledMatrix.Fill(Color.Black);
}